In [1]:
from model_lomba import *

model = switching_model(1,2,5,2)

#Di sini satuan waktunya dalam menit
num_of_steps = 24*60 # 1 day
for i in range(num_of_steps):
    model.step()

(14, 1)
(14, 2)
(14, 3)
(13, 3)
(12, 3)
(13, 3)
(12, 3)
(12, 2)
(12, 1)
(13, 1)
(12, 1)
(13, 1)
(12, 1)
(13, 1)
(12, 1)
(12, 2)
(11, 2)
(10, 2)
(9, 2)
(9, 3)
(9, 2)
(8, 2)
(8, 3)
(9, 3)
(10, 3)
(10, 4)
(11, 4)
(10, 4)
(9, 4)
(8, 4)
(8, 3)
(8, 2)
(8, 3)
(9, 3)
(9, 4)
(9, 5)
(9, 6)
(9, 5)
(9, 6)
(8, 6)
(7, 6)
(6, 6)
(6, 5)
(5, 5)
(5, 6)
(5, 5)
(4, 5)
(4, 4)
(4, 5)
(3, 5)
(3, 4)
(3, 5)
(3, 4)
(3, 3)
(3, 4)
(4, 4)
(4, 3)
(5, 3)
(4, 3)
(4, 4)
(3, 4)
(2, 4)
(1, 4)
(0, 4)
(1, 4)
(1, 3)
(1, 4)
(1, 3)
(1, 2)
(1, 1)
(0, 1)
(0, 2)
(0, 3)
(0, 2)
(0, 3)
(0, 2)
(0, 1)
(0, 2)
(1, 2)
(2, 2)
(2, 1)
(2, 2)
(1, 2)
(0, 2)
(0, 1)
(19, 1)
(19, 2)
(18, 2)
(17, 2)
(18, 2)
(17, 2)
(18, 2)
(19, 2)
(0, 2)
(1, 2)
(0, 2)
(0, 3)
(0, 2)
(1, 2)
(1, 3)
(0, 3)
(0, 2)
(0, 1)
(0, 0)
(19, 0)
(18, 0)
(18, 19)
(18, 18)
(19, 18)
(19, 17)
(19, 18)
(18, 18)
(18, 17)
(18, 16)
(19, 16)
(18, 16)
(17, 16)
(17, 15)
(17, 14)
(17, 13)
(17, 14)
(16, 14)
(17, 14)
(18, 14)
(18, 13)
(18, 12)
(17, 12)
(18, 12)
(18, 13)
(18, 12)
(18, 11)
(

In [2]:
#Ambil data agent
agent_data = model.datacollector.get_agent_vars_dataframe()
agent_data.head()

Position
Step AgentID         
0    0           None
     1           None
     2           None
     3           None
     4           None

In [3]:
#Ambil data motorist
motorist_pos = agent_data.xs(model.motorists[0].unique_id, level = "AgentID")["Position"]
motorist_pos.head()

Step
0    (13, 1)
1    (14, 1)
2    (14, 2)
3    (14, 3)
4    (13, 3)
Name: Position, dtype: object